# Multiple of 32

In [28]:
for i in range(25):
    x = 32 * i
    y = x * 3/4
    print("X: ", x,  "_" * 8, "Y: ", int(y))

X:  0 ________ Y:  0
X:  32 ________ Y:  24
X:  64 ________ Y:  48
X:  96 ________ Y:  72
X:  128 ________ Y:  96
X:  160 ________ Y:  120
X:  192 ________ Y:  144
X:  224 ________ Y:  168
X:  256 ________ Y:  192
X:  288 ________ Y:  216
X:  320 ________ Y:  240
X:  352 ________ Y:  264
X:  384 ________ Y:  288
X:  416 ________ Y:  312
X:  448 ________ Y:  336
X:  480 ________ Y:  360
X:  512 ________ Y:  384
X:  544 ________ Y:  408
X:  576 ________ Y:  432
X:  608 ________ Y:  456
X:  640 ________ Y:  480
X:  672 ________ Y:  504
X:  704 ________ Y:  528
X:  736 ________ Y:  552
X:  768 ________ Y:  576


# Converting Yolo to ncnn 

In [ ]:
#!pip install ultralytics

from ultralytics import YOLO

model = YOLO("yolo11n.pt")

# export the model for a lower input resolution/shape 320 for performance improvement
model.export(format="ncnn", imgsz=(256, 256))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 25.6MB/s]


Ultralytics 8.3.68 🚀 Python-3.9.21 torch-2.5.1 CPU (Cortex-A76)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 256, 256) BCHW and output shape(s) (1, 84, 1344) (5.4 MB)

TorchScript: starting export with torch 2.5.1...
TorchScript: export success ✅ 3.5s, saved as 'yolo11n.torchscript' (10.5 MB)

NCNN: starting export with NCNN 1.0.20241226...
NCNN: WARNING ⚠️ PNNX not found. Attempting to download binary file from https://github.com/pnnx/pnnx/.
Note PNNX Binary file must be placed in current working directory or in /home/vesper_pi/miniconda3/envs/surveillance/lib/python3.9/site-packages/ultralytics. See PNNX repo for full installation instructions.
NCNN: successfully found latest PNNX asset file pnnx-20241223-linux-aarch64.zip


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19.9M/19.9M [00:00<00:00, 26.6MB/s]
Unzipping pnnx-20241223-linux-aarch64.zip to /home/vesper_pi/surveillance_system/pipeline/pnnx-20241223-linux-aarch64...: 100%|██████████| 3/3 [00:00<00:00,  7.07fi

NCNN: running '/home/vesper_pi/miniconda3/envs/surveillance/lib/python3.9/site-packages/ultralytics/pnnx yolo11n.torchscript ncnnparam=yolo11n_ncnn_model/model.ncnn.param ncnnbin=yolo11n_ncnn_model/model.ncnn.bin ncnnpy=yolo11n_ncnn_model/model_ncnn.py pnnxparam=yolo11n_ncnn_model/model.pnnx.param pnnxbin=yolo11n_ncnn_model/model.pnnx.bin pnnxpy=yolo11n_ncnn_model/model_pnnx.py pnnxonnx=yolo11n_ncnn_model/model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 256, 256]"'



pnnxparam = yolo11n_ncnn_model/model.pnnx.param
pnnxbin = yolo11n_ncnn_model/model.pnnx.bin
pnnxpy = yolo11n_ncnn_model/model_pnnx.py
pnnxonnx = yolo11n_ncnn_model/model.pnnx.onnx
ncnnparam = yolo11n_ncnn_model/model.ncnn.param
ncnnbin = yolo11n_ncnn_model/model.ncnn.bin
ncnnpy = yolo11n_ncnn_model/model_ncnn.py
fp16 = 0
optlevel = 2
device = cpu
inputshape = [1,3,256,256]f32
inputshape2 = 
customop = 
moduleop = 
get inputshape from traced inputs
inputshape = [1,3,256,256]f32
Error in cpuinfo: prctl(PR_SVE_GET_VL) failed
############# pass_level0
inline module = torch.nn.modules.linear.Identity
inline module = ultralytics.nn.modules.block.Attention
inline module = ultralytics.nn.modules.block.Bottleneck
inline module = ultralytics.nn.modules.block.C2PSA
inline module = ultralytics.nn.modules.block.C3k
inline module = ultralytics.nn.modules.block.C3k2
inline module = ultralytics.nn.modules.block.DFL
inline module = ultralytics.nn.modules.block.PSABlock
inline module = ultralytics.nn.m

NCNN: export success ✅ 4.6s, saved as 'yolo11n_ncnn_model' (10.0 MB)

Export complete (9.3s)
Results saved to /home/vesper_pi/surveillance_system/pipeline
Predict:         yolo predict task=detect model=yolo11n_ncnn_model imgsz=256  
Validate:        yolo val task=detect model=yolo11n_ncnn_model imgsz=256 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'yolo11n_ncnn_model'

# Convert edgeface from pytorch to onnx. 

### After many tries I gave up to convert it to ncnn cause it has to many layers not available in ncnn.

Using pytorch (which has the model in its repository) to export model to onnx

In [ ]:
import torch

model_name = "edgeface_s_gamma_05"
model = torch.hub.load('otroshi/edgeface', model_name, source='github', pretrained=True)
model.eval()

dummy_input = torch.randn(1, 3, 112, 112)

onnx_file_path = f"{model_name}.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_file_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"},
    },
)

print(f"Model has been exported to ONNX format at {onnx_file_path}.")

Using cache found in /home/vesper_pi0/.cache/torch/hub/otroshi_edgeface_main


Model has been exported to ONNX format at edgeface_s_gamma_05.onnx.
